In [53]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [56]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-07-19 09:41:37--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M   163MB/s    in 0.2s    

2024-07-19 09:41:37 (163 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [114]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


# data cleaning
df_books['isbn'] = pd.to_numeric(df_books['isbn'], errors="coerce")
df_books.dropna(subset=['isbn'], inplace=True)

df_ratings['isbn'] = pd.to_numeric(df_ratings['isbn'], errors="coerce")
df_ratings.dropna(subset=['isbn'], inplace=True)

# drop users with less than 200 ratings
user_counts = df_ratings['user'].value_counts()
valid_users = user_counts[user_counts >= 200].index
df_ratings = df_ratings[df_ratings['user'].isin(valid_users)]

# drop books with less than 100 ratings
book_counts = df_ratings['isbn'].value_counts()
valid_books = book_counts[book_counts >= 100].index
df_ratings = df_ratings[df_ratings['isbn'].isin(valid_books)]
df_books = df_books[df_books['isbn'].isin(valid_books)]

In [236]:
# prepare df_ratings for nearest neighbors algorithm

ratings_matrix = df_ratings.pivot_table(index='isbn', columns='user', values='rating').fillna(0)
#print(ratings_matrix.shape)
n = 6
nbrs = NearestNeighbors(n_neighbors=n, algorithm='auto').fit(ratings_matrix)


In [240]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    isbn = df_books[df_books['title'] == book]['isbn'].iloc[0]

    if isbn in ratings_matrix.index:
        target_vector = ratings_matrix.loc[isbn].values.reshape(1, -1)
        dist, recommended_books_indices = nbrs.kneighbors(target_vector)
        recommended_books_isbn = ratings_matrix.index[recommended_books_indices[0]]
        recommended_books = df_books[df_books['isbn'].isin(recommended_books_isbn)].copy()
        recommended_books.loc[:, 'isbn'] = pd.Categorical(recommended_books['isbn'], categories=recommended_books_isbn, ordered=True)
        recommended_books = recommended_books.sort_values('isbn')
        recommended_books_titles = np.array(recommended_books['title'])
        output = pd.DataFrame({
            'title'   : recommended_books_titles,
            'distance': dist[0]
          }).head(5).values

        return [book, output]

    else:
        print(f"{book} not found in the ratings matrix.")


In [241]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
      print(2)
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
      print(3)
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

ValueError: X has 1 features, but NearestNeighbors is expecting 770 features as input.